<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

## GroupBy Base Model using TPOT

Objective

- Leverage `pandas_profiling` for fast data understanding
- Practice data preprocessing
- Establish a baseline Model using Logistic Regression, XGBoostClassifier etc
- Interpret black box models with SHAP
- Generate optimal pipeline for classification task using AutoML

##  Load data

In [3]:
import pandas as pd
train_df = pd.read_csv('../dat/upliftfull.csv')# YOUR CODE HERE

In [4]:
train_df# YOUR CODE HERE

,recency,history,used_discount,used_bogo,is_referral,treatment,target,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,142.44,1,0,0,-1,0,0,1,0,0,1,0
1,6,329.08,1,1,1,0,0,1,0,0,0,0,1
2,7,180.65,0,1,1,-1,0,0,1,0,0,0,1
3,9,675.83,1,0,1,1,0,1,0,0,0,0,1
4,2,45.34,1,0,0,-1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,0,1,0,0,0,1,0,0,1
63996,5,38.91,0,1,1,1,0,0,0,1,0,1,0
63997,6,29.99,1,0,1,1,0,0,0,1,0,1,0
63998,1,552.94,1,0,1,-1,0,0,1,0,1,0,0


In [5]:
train_df.shape# YOUR CODE HERE

(64000, 13)

In [7]:
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [78]:
# train_df_sample= train_df.sample(10000, random_state=3)

In [8]:

profile = ProfileReport(train_df, title="Pandas Profiling Electronics Purchase Report", html={'style': {'full_width': True}}, sort=None)# YOUR CODE HERE

In [9]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [10]:
%%time
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 21.4 s, sys: 23.5 s, total: 44.8 s
Wall time: 30.2 s


 Pandas profiling Profile reports that there are 2 numeric variables and 11 categorical variables.There's also some level of imbalance with the target variable.

In [11]:
train_df.dtypes 

recency                   int64
history                 float64
used_discount             int64
used_bogo                 int64
is_referral               int64
treatment                 int64
target                    int64
zip_code_Rural            int64
zip_code_Surburban        int64
zip_code_Urban            int64
channel_Multichannel      int64
channel_Phone             int64
channel_Web               int64
dtype: object

## Preprocess Data

In [14]:
from sklearn.model_selection import train_test_split

In [29]:
X = train_df.drop(['target'],axis=1)
y = train_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=train_df['target'])

## Build Logistic Regression and XGBoost models

In [30]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
# from sklearn.svm import LinearSVC

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score as accuracy,
    recall_score as recall,
    precision_score as precision,
    f1_score
)

In [31]:
%%time
lr_model = LogisticRegression(random_state=0).fit(X_train, y_train)
# YOUR CODE HERE
lr_pred = lr_model.predict(X_test)

CPU times: user 1.58 s, sys: 1.75 s, total: 3.33 s
Wall time: 445 ms


2. Calculate these classification metrics for the Logistic Regression model:

    * accuracy
    * precision
    * recall
    * f1 score
    * confusion matrix

In [32]:
print("accuracy:", accuracy(y_test, lr_pred))
print("precision:", precision(y_test, lr_pred))
print("recall:", recall(y_test, lr_pred))
print("f1 score:", f1_score(y_test, lr_pred))
print("confusion matrix:")
print(confusion_matrix(y_test, lr_pred))

accuracy: 0.8533125
precision: 1.0
recall: 0.00042589437819420784
f1 score: 0.0008514261387824605
confusion matrix:
[[13652     0]
 [ 2347     1]]


3. Instantiate, train and predict using the XGBoost model. 

In [33]:
from xgboost import XGBClassifier

In [34]:
%%time
xgb_model = XGBClassifier() 
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

CPU times: user 24.3 s, sys: 0 ns, total: 24.3 s
Wall time: 3.18 s


In [35]:
#Results
print("accuracy:", accuracy(y_test, xgb_pred))
print("precision:", precision(y_test, xgb_pred))
print("recall:", recall(y_test, xgb_pred))
print("f1 score:", f1_score(y_test, xgb_pred))
print("confusion matrix:")
print(confusion_matrix(y_test, xgb_pred))

accuracy: 0.8513125
precision: 0.26153846153846155
recall: 0.007240204429301534
f1 score: 0.014090343970161626
confusion matrix:
[[13604    48]
 [ 2331    17]]


In [36]:
from utils import display_metrics
# display_metrics(y_test, lr_pred)
display_metrics(y_test, lr_pred)
# YOUR CODE HERE

Confusion Matrix: 
[[13652     0]
 [ 2347     1]]
Accuracy: 0.853
Recall: 0.000
Precision: 1.000
F1 Score: 0.001
